In [1]:
!pip install albumentations opencv-python

   ---------------------------------------- 0.0/39.4 MB ? eta -:--:--
   -- ------------------------------------- 2.4/39.4 MB 11.2 MB/s eta 0:00:04
   ----------- ---------------------------- 11.8/39.4 MB 29.5 MB/s eta 0:00:01
   ------------- -------------------------- 13.6/39.4 MB 23.8 MB/s eta 0:00:02
   --------------- ------------------------ 15.7/39.4 MB 19.4 MB/s eta 0:00:02
   ------------------ --------------------- 18.1/39.4 MB 17.3 MB/s eta 0:00:02
   -------------------- ------------------- 20.4/39.4 MB 16.1 MB/s eta 0:00:02
   ---------------------- ----------------- 22.5/39.4 MB 15.3 MB/s eta 0:00:02
   ------------------------ --------------- 24.4/39.4 MB 14.8 MB/s eta 0:00:02
   ------------------------------ --------- 30.4/39.4 MB 16.2 MB/s eta 0:00:01
   ---------------------------------- ----- 34.3/39.4 MB 16.4 MB/s eta 0:00:01
   -------------------------------------- - 37.7/39.4 MB 16.3 MB/s eta 0:00:01
   ---------------------------------------  39.3/39.4 MB 16.7 

In [3]:
import os
import cv2
import albumentations as A
from tqdm import tqdm

# Paths
image_dir = "../dataset/images/test"
label_dir = "../dataset/labels/test"
aug_img_dir = "../dataset/images/aug_medellin"
aug_lbl_dir = "../dataset/labels/aug_medellin"

transform = A.Compose([
    A.RandomBrightnessContrast(p=0.7),
    A.MotionBlur(blur_limit=3, p=0.4),
    A.Rotate(limit=15, p=0.5),
    A.RandomScale(scale_limit=0.3, p=0.5),
    A.Perspective(scale=(0.05, 0.1), p=0.4),
    A.HorizontalFlip(p=0.5),
    A.RandomSizedBBoxSafeCrop(height=640, width=640, p=0.3),
    A.ColorJitter(p=0.3)
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

# Número de augmentations por imagen
N = 30

In [4]:
for filename in tqdm(os.listdir(image_dir)):
    if not filename.endswith(".jpg"):
        continue

    base_name = os.path.splitext(filename)[0]
    img_path = os.path.join(image_dir, filename)
    label_path = os.path.join(label_dir, base_name + ".txt")

    # Leer imagen
    image = cv2.imread(img_path)
    h, w = image.shape[:2]

    # Leer anotaciones en formato YOLO
    bboxes = []
    class_labels = []
    with open(label_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            cls = int(parts[0])
            x, y, bw, bh = map(float, parts[1:])
            bboxes.append([x, y, bw, bh])
            class_labels.append(cls)

    for i in range(N):
        transformed = transform(image=image, bboxes=bboxes, class_labels=class_labels)

        aug_img = transformed["image"]
        aug_bboxes = transformed["bboxes"]
        aug_labels = transformed["class_labels"]

        # Guardar imagen
        out_img_name = f"{base_name}_aug_{i}.jpg"
        out_img_path = os.path.join(aug_img_dir, out_img_name)
        cv2.imwrite(out_img_path, aug_img)

        # Guardar anotaciones
        out_lbl_path = os.path.join(aug_lbl_dir, f"{base_name}_aug_{i}.txt")
        with open(out_lbl_path, "w") as f:
            for box, cls in zip(aug_bboxes, aug_labels):
                f.write(f"{cls} {' '.join(f'{coord:.6f}' for coord in box)}\n")

100%|██████████| 22/22 [00:13<00:00,  1.62it/s]
